In [1]:
import ee
import geemap


ee.Authenticate()
ee.Initialize(project='raman-461708')
# Initialize the Earth Engine API (run ee.Authenticate() once before this in a new environment)


wri_data = ee.FeatureCollection("projects/raman-461708/assets/class_data")
roi = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

india_wri_data = wri_data.filterBounds(roi)


NG = india_wri_data.filter(ee.Filter.eq('class_labe', 'Natural/Semi-natural grassland'));
CG = india_wri_data.filter(ee.Filter.eq('class_labe', 'Cultivated grassland'));
OL = india_wri_data.filter(ee.Filter.eq('class_labe', 'Other land cover'));

# Create a geemap map
m = geemap.Map(center=[22, 80], zoom=4)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")

m.addLayer(NG, {'color': 'blue'}, 'Natural/Semi-natural grassland');
m.addLayer(CG, {'color': 'red'}, 'Cultivated grassland');
m.addLayer(OL, {'color': 'green'}, 'Other land cover');

# Display the map (in a Jupyter notebook)
m

Map(center=[22, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [6]:
def get_lulc_test_band_ours_band(exp):
    if exp == "OURS":
        lulc_ours = ee.Image(
        "projects/raman-461708/assets/LULC_v4_PanIndia_2023-07-01_2024-06-30"
        )
        # CHANGE THESE to the actual band names in your assets
        
        OURS_BAND = 'predicted_label'          # band in lulc_ours with classes 8–12
        SCRUB_LIST = [12, 7, 6]
        return OURS_BAND, lulc_ours, SCRUB_LIST
    
    elif exp == "WC":
        # -------------------------------------------------------------
        # 1. INPUTS
        # -------------------------------------------------------------

 


        lulc_ours = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi)
        OURS_BAND = 'Map'
        SCRUB_LIST = [20]
        return OURS_BAND, lulc_ours, SCRUB_LIST
    
    elif exp == "ESRI":
        # ESRI LULC collection
        COLLECTION_ID = "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"

        # Original → remapped classes (same as your JS CONFIG)
        REMAP_FROM = [1, 2, 4, 5, 7, 8, 9, 10, 11]
        REMAP_TO   = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        # After remap:
        # 4 = Crops
        # 9 = Rangeland

        esri_lulc = ee.ImageCollection(COLLECTION_ID)

        def get_esri_lulc_year(year):
            """Return remapped ESRI LULC composite for a given year."""
            start = ee.Date.fromYMD(year, 1, 1)
            end   = ee.Date.fromYMD(year, 12, 31)

            img = esri_lulc.filterDate(start, end).mosaic()
            # Remap first band; name it 'lulc'
            remapped = img.remap(REMAP_FROM, REMAP_TO, 0)
            return remapped.rename('lulc')

        # Example: get 2020 map
        year = 2022
        lulc_ours = get_esri_lulc_year(year)
        OURS_BAND = 'lulc'
        SCRUB_LIST = [2,9,6]
        return OURS_BAND, lulc_ours, SCRUB_LIST
    elif exp == "DW_O":
        lulc_ours = ic.select("b1").mosaic().clip(roi)
        OURS_BAND = 'b1'
        SCRUB_LIST = [7,12]

        return OURS_BAND, lulc_ours, SCRUB_LIST
    elif exp == "DW_M":
        lulc_ours = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate('2023-01-01', '2023-12-31').mode().clip(roi)
        OURS_BAND = 'label'
        SCRUB_LIST = [7]

        return OURS_BAND, lulc_ours, SCRUB_LIST
    elif exp == "WRI":
        lulc_ours = ee.ImageCollection("projects/global-pasture-watch/assets/ggc-30m/v1/grassland_c").mosaic().clip(roi)
        OURS_BAND = "dominant_class"
        SCRUB_LIST = [2]
        return OURS_BAND, lulc_ours, SCRUB_LIST
        
TODO = "WC"    
OURS_BAND, lulc_ours, SCRUB_LIST = get_lulc_test_band_ours_band(TODO)


In [7]:
 #-------------------------------------------------------------------
# 3. Set reference label: for NG points, ref = 1 (scrub)
# -------------------------------------------------------------------
def set_reference_label(feature):
    # All NG points are scrub in reference
    return feature.set('ref', 1)

NG_points_ref = NG.map(set_reference_label)

# -------------------------------------------------------------------
# 4. Sample LULC at NG point locations
# -------------------------------------------------------------------
sampled = lulc_ours.select(OURS_BAND).sampleRegions(
    collection=NG_points_ref,
    scale=10,
    geometries=False
)

# -------------------------------------------------------------------
# 5. Convert predicted_label to binary: scrub (1) vs rest (0)
# -------------------------------------------------------------------
def set_predicted_binary(feature):
    pred_class = ee.Number(feature.get(OURS_BAND))
    pred_binary = ee.Algorithms.If(
        ee.List(SCRUB_LIST).contains(pred_class),
        1,  # predicted scrub
        0   # predicted rest
    )
    return feature.set('pred', pred_binary)

sampled_bin = sampled.map(set_predicted_binary)

# -------------------------------------------------------------------
# 6. Build confusion matrix on NG points only
#    ref: 'ref'   (always 1 here: scrub)
#    pred: 'pred' (0 = rest, 1 = scrub)
# -------------------------------------------------------------------
cm = sampled_bin.errorMatrix('ref', 'pred')

print("Confusion Matrix on NG points (rows = reference, cols = prediction):")
print(cm.getInfo())

print("\nOverall Accuracy (on NG only):")
print(cm.accuracy().getInfo())

print("\nProducer's Accuracy (per reference class [0,1]):")
print(cm.producersAccuracy().getInfo())

print("\nUser's (Consumer's) Accuracy (per predicted class [0,1]):")
print(cm.consumersAccuracy().getInfo())

Confusion Matrix on NG points (rows = reference, cols = prediction):
[[0, 0], [5879, 1674]]

Overall Accuracy (on NG only):
0.22163378789884813

Producer's Accuracy (per reference class [0,1]):
[[0], [0.22163378789884813]]

User's (Consumer's) Accuracy (per predicted class [0,1]):
[[0, 1]]


In [27]:
NG.size()

In [30]:
NG_points_ref.size()

In [32]:
sampled.size()

In [8]:
NG.limit(1).getInfo()

{'type': 'FeatureCollection',
 'columns': {'class': 'Long',
  'class_labe': 'String',
  'imagery': 'String',
  'ref_date': 'Long',
  'system:index': 'String',
  'x': 'Float',
  'y': 'Float',
  'year': 'Integer'},
 'version': 1753120131124869,
 'id': 'projects/raman-461708/assets/class_data',
 'properties': {'system:asset_size': 61580206},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [80.06636104917712, 35.060248338552]},
   'id': '00010000000000006cb3',
   'properties': {'class': 2,
    'class_labe': 'Natural/Semi-natural grassland',
    'imagery': 'Google',
    'ref_date': 1096873200000,
    'x': 80.06636,
    'y': 35.06025,
    'year': 2004}}]}